To Do's: 

- zwei code blocks, die mit listen arbeiten in arrays umwandeln -> performanter

Books: 

https://www.taylorfrancis.com/books/edit/10.1201/9780367631888/recommender-systems-pavan-kumar-vairachilai-sirisha-potluri-sachi-nandan-mohanty

https://beluga.sub.uni-hamburg.de/vufind/Record/1656091313?rank=1

In [2]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from surprise import Reader, Dataset
from surprise import KNNBasic, KNNWithMeans, SVD, CoClustering, SlopeOne, SVDpp, NMF, BaselineOnly
from surprise.model_selection import KFold, RepeatedKFold, cross_validate, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
import time
from numpy.random import permutation

### Import data

In [3]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";") 
#df = [~df["category"].isin(["Betriebssystem", "Dienste", "Einsatzfelder / Erfahrungen / Schwerpunkte", "Rolle (intern)"])]
df.head(5)

,person,skill,category
0,12,Windows NT/2000/XP,Betriebssystem
1,12,MAC OS X,Betriebssystem
2,12,Windows 7,Betriebssystem
3,12,Windows 8,Betriebssystem
4,12,iOS,Betriebssystem


In [4]:
data = df

Code kann zunächst auskommentiert werden, da die binäre Matrix als csv abgespeichert wurde, um die Laufzeit des Notebooks während der Entwicklung zu reduzieren.

In [5]:
employee_dict = {}
for n in range(1,data["person"].nunique()+1):
    employee_dict[n] = data["person"].unique()[n-1]

In [6]:
# So later functions work, it makes sense to rename persons from 1 to maxno. 
# Create dictionary which matches the employee
persons = data["person"].unique()
#np.arange(1,len(data["person"].unique())+1)
skills = sorted(data["skill"].unique())

In [7]:
# skills_per_person = []
# for person in data["person"].unique(): 
#     skillset = []
#     for skill in skills:
#         if skill in data[data["person"] == person]["skill"].unique():
#             skillset.append(1)
#         else:
#             skillset.append(0)
#     skills_per_person.append(skillset)


In [8]:
# matrix = pd.DataFrame(index=persons,columns=skills,data=skills_per_person)

Als csv. speichern

In [9]:
# matrix.to_csv("data/origin_binary_matrix.csv", index = False)

Csv. importieren

In [10]:
matrix = pd.read_csv("data/origin_binary_matrix.csv")

In [11]:
matrix

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Recommender 

### Preprocessing <br>
First we create a matrix, that only contains the skills (items)

In [12]:
data_skills  = matrix
data_skills 

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In the next step we will compute the item-item relationships of our skills. Our final goal here is to construct a new item by item matrix containing the weights (relationships) between each of our skills where a perfect correlation equals 1 and no correlation at all equals 0.
<br>
In order to do so, we will first normalize the user vectors. The idea behind this approach is, that a user with many skills contributes less to any individual skill. For example if a user, that rules only 3 skills, rules a skill X it is more valueable than if a user, that rules 20 skills, that particular skill.

* First we caclulat the magnitude for every user

In [13]:
# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_skills).sum(axis=1))

In [14]:
magnitude

0       8.774964
1       8.602325
2       5.567764
3       5.567764
4       1.000000
         ...    
399     2.828427
400     4.358899
401     3.000000
402     2.645751
403    12.206556
Length: 404, dtype: float64

* Now we use a users magnitude to normalize the ratings of this corresponding user

In [15]:
data_skills_row_norm=  data_skills.divide(magnitude, axis='index')

In [16]:
data_skills_row_norm

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.000000,0.116248,0.000000,0.000000,0.116248,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.000000,0.179605,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.179605,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
400,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
401,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
402,0.0,0.377964,0.377964,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


* Now we calculate the cosine simularity to express the relationship between the items.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [18]:
def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix


In [19]:
data_matrix = calculate_similarity(data_skills_row_norm)


In [20]:
data_matrix

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
.NET Compact Framework,1.000000,0.239803,0.517252,0.076796,0.000000,0.385593,0.014963,0.000000,0.052473,0.130239,...,0.000000,0.000000,0.118497,0.020051,0.000000,0.000000,0.082602,0.000000,0.148524,0.039486
.NET Core,0.239803,1.000000,0.399014,0.000000,0.000000,0.452524,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012137,0.102612,0.008383,0.000000
.NET Framework,0.517252,0.399014,1.000000,0.118449,0.000000,0.351258,0.049707,0.055426,0.050197,0.139407,...,0.000000,0.041472,0.104624,0.048462,0.000000,0.011269,0.158188,0.123404,0.165101,0.031949
3D-Modellierung,0.076796,0.000000,0.118449,1.000000,0.067836,0.040880,0.042217,0.025233,0.034520,0.055505,...,0.000000,0.000000,0.203767,0.147701,0.022892,0.018698,0.206958,0.015334,0.338931,0.092357
ABAP,0.000000,0.000000,0.000000,0.067836,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.113540,0.000000,0.000000,0.047097,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
visual paradigm,0.000000,0.000000,0.011269,0.018698,0.113540,0.000000,0.000000,0.098304,0.000000,0.000000,...,0.000000,0.000000,0.059097,0.000000,0.000000,1.000000,0.018618,0.000000,0.012859,0.000000
vnc,0.082602,0.012137,0.158188,0.206958,0.000000,0.088503,0.146734,0.035497,0.108038,0.125648,...,0.000000,0.000000,0.480757,0.232385,0.084432,0.018618,1.000000,0.094197,0.326403,0.289718
vs code,0.000000,0.102612,0.123404,0.015334,0.000000,0.033641,0.000000,0.000000,0.000000,0.065258,...,0.000000,0.000000,0.114259,0.034019,0.050780,0.000000,0.094197,1.000000,0.083808,0.049125
xHTML,0.148524,0.008383,0.165101,0.338931,0.047097,0.137666,0.094768,0.024518,0.074621,0.132712,...,0.092732,0.000000,0.281336,0.310003,0.099723,0.012859,0.326403,0.083808,1.000000,0.348152


* Let's have a look, what are the most simular items to windows 10 and python?

In [21]:
print (data_matrix.loc['Windows 10'].nlargest(5))

Windows 10    1.000000
Ubuntu        0.330780
Englisch      0.326022
Gitlab        0.325900
MySQL         0.323192
Name: Windows 10, dtype: float64


In [22]:
print(data_matrix.loc['Python'].nlargest(5))

Python                           1.000000
GIT                              0.484364
Java                             0.483214
Test Driven Development (TDD)    0.474858
Softwaretests                    0.460265
Name: Python, dtype: float64


## User-Item
To perform user recommendations, we look at the users known skills and their corresponding simularities. Basically, we want to create a score for each skill in our dataset for that user and then we can simply choose the n items with the highest score.
<br>
* Die ausführungen beziehen sich auf folgenden Artikel (https://medium.com/radon-dev/item-item-collaborative-filtering-with-binary-or-unary-data-e8f0b465b2c3)

1.  Get the known skills of a user

In [23]:
user = 1
known_user_skills = data_skills.loc[user]
known_user_skills = known_user_skills[known_user_skills >0].index.values

In [24]:
known_user_skills

array(['.NET Framework', 'ADO.NET', 'ASP.NET', 'ASP.NET MVC-Framework',
       'ASP.NET WebAPI', 'Active Directory', 'Ajax', 'Android',
       'Angular (2 und höher)', 'Aspektorientierte Programmierung (AOP)',
       'Azure DevOps', 'BPMN', 'Behaviour Driven Development (BDD)', 'C#',
       'CSS', 'Continuous Integration (CI)', 'Dependency Injection',
       'Design Pattern', 'Englisch', 'Entity Framework', 'Entwurfsmuster',
       'FileZilla', 'GIT', 'GraphQL', 'HTML', 'HTML5', 'ITIL',
       'Implementierung', 'IntelliJ(Idea)',
       'Internet Information Services (IIS)', 'JIRA', 'JSON', 'Java',
       'JavaScript', 'KANBAN', 'LINQ', 'MS Project',
       'Microsoft SQL Server', 'Microsoft Visio', 'NUnit',
       'Objektrelationale Abbildung (ORM)', 'Open SSH', 'PostgreSQL',
       'PowerShell', 'Python', 'REST', 'React', 'Ruby', 'SQL',
       'Softwaredesign/-architektur', 'Softwaretests',
       'Statische Codeanalyse', 'Subversion', 'T-SQL',
       'Team Foundation Server (TF Serv

2. Get the rating for all items of a user as a sparse vector

In [25]:
# Users likes for all items as a sparse vector.
user_skills_vector = data_skills_row_norm.loc[user]
user_skills_vector

.NET Compact Framework    0.000000
.NET Core                 0.000000
.NET Framework            0.116248
3D-Modellierung           0.000000
ABAP                      0.000000
                            ...   
visual paradigm           0.000000
vnc                       0.000000
vs code                   0.000000
xHTML                     0.000000
xtCommerce                0.000000
Name: 1, Length: 735, dtype: float64

3. "So we get the score for user u and item i by summing together all the weights for that item Wij multiplied with the users rating for that item rui. We then divide by the sum of all the weights for that item Wi."

In [26]:
score = data_matrix.dot(user_skills_vector).div(data_matrix.sum(axis=1))
score

.NET Compact Framework    0.028545
.NET Core                 0.037522
.NET Framework            0.037639
3D-Modellierung           0.027076
ABAP                      0.016506
                            ...   
visual paradigm           0.025104
vnc                       0.023201
vs code                   0.021966
xHTML                     0.027663
xtCommerce                0.021771
Length: 735, dtype: float64

4. Remove known skills from the recommendation

In [27]:
score = score.drop(known_user_skills)

5 Print the n-largest scores

In [28]:
print (score.nlargest(5))

EventModeling    0.060727
MSBuild          0.042007
gdscript         0.040422
Kendo UI         0.038438
Windows Forms    0.037974
dtype: float64


### Including Neighborhood
"Depending on the size of our dataset it might be inefficient to do scoring for all items for every user. Chances are we’re only interested in the score for the top 5 or 10 items and don’t really care about the score for the 500th one.
So instead we can limit our calculations to a neighborhood of only the n most similar items to the users own ratings and then do the scoring on those"

1. For the neighborhood approach we sort the simularities of our simularity matrix in descending order for every skill. (order: most simular skill --> least simular skill)

In [29]:
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
for i in range(0, len(data_matrix.columns)):
    data_neighbours.iloc[i,:10] = data_matrix.iloc[0:,i].sort_values(ascending=False)[:10].index

data_neighbours

,1,2,3,4,5,6,7,8,9,10
.NET Compact Framework,.NET Compact Framework,.NET Framework,ADO.NET,Windows Communication Foundation (WCF),Infragistics,BizTalk,.NET Core,Microsoft Enterprise Library,Sonstiges (Freitextfeld),Windows Identity Foundation (WIF)
.NET Core,.NET Core,Infragistics,BizTalk,ADO.NET,Windows Communication Foundation (WCF),.NET Framework,Windows Presentation Foundation (WPF),MSBuild,Sharepoint Entwicklung,.NET Compact Framework
.NET Framework,.NET Framework,LINQ,NUnit,Windows Forms,.NET Compact Framework,Entity Framework,ASP.NET WebAPI,ASP.NET,Windows Presentation Foundation (WPF),Team Foundation Server (TF Server)
3D-Modellierung,3D-Modellierung,Adobe Premiere,After Effects,Adobe Indesign,OpenGL,JSON,CSS,Angular Generalist,Englisch,HTML5
ABAP,ABAP,Apache JMeter,SAP NWDS,Infinispan,ProxySQL,WebDynpro,Spring-Integration,Sonatype Nexus,SAP NWDI,RedDot
...,...,...,...,...,...,...,...,...,...,...
visual paradigm,visual paradigm,BaaN/Infor ERP LN,Selectica,MS Project,First Spirit,BPMN,Rabbitmq,Spring-Integration,Management Consulting,Dynamics NAV/Navision
vnc,vnc,Open SSH,samba,SUSE Linux,OpenOffice / LibreOffice,SSL,CVS,Apache,TCP/IP,Postfix
vs code,vs code,helm,NFS,Ansible,UWP,NEOS,Firewall (IP Tables),Raspberry Pi,BashScript,Testing Library
xHTML,xHTML,Ajax,HTML,jQuery,Subversion,JSON,HTML5,XML/XSL,Windows NT/2000/XP,CSS


In [30]:
user = 1
known_user_skills = data_skills.loc[user]
known_user_skills = known_user_skills[known_user_skills >0].index.values

2. Now we build the neighborhood from the most simular items to the ones that our user already rules.

In [31]:
most_similar_to_rules_skills = data_neighbours.loc[known_user_skills]
most_similar_to_rules_skills

,1,2,3,4,5,6,7,8,9,10
.NET Framework,.NET Framework,LINQ,NUnit,Windows Forms,.NET Compact Framework,Entity Framework,ASP.NET WebAPI,ASP.NET,Windows Presentation Foundation (WPF),Team Foundation Server (TF Server)
ADO.NET,ADO.NET,BizTalk,Windows Communication Foundation (WCF),Infragistics,.NET Core,Microsoft Enterprise Library,Entity Framework,.NET Compact Framework,ASP.NET WebAPI,ASP.NET MVC-Framework
ASP.NET,ASP.NET,LINQ,NUnit,Entity Framework,ASP.NET WebAPI,Windows Forms,Team Foundation Server (TF Server),ASP.NET MVC-Framework,SQL,.NET Framework
ASP.NET MVC-Framework,ASP.NET MVC-Framework,ASP.NET WebAPI,LINQ,Entity Framework,NUnit,NHibernate,ASP.NET,Windows Presentation Foundation (WPF),Jakarta Commons,Kendo UI
ASP.NET WebAPI,ASP.NET WebAPI,Entity Framework,LINQ,NUnit,ASP.NET MVC-Framework,ASP.NET,Team Foundation Server (TF Server),.NET Framework,NHibernate,Windows Presentation Foundation (WPF)
...,...,...,...,...,...,...,...,...,...,...
XML/XSL,XML/XSL,Implementierung,SOAP,REST,MySQL,JSON,Subversion,UML,Unit Tests (Komponententests),Entwurfsmuster
iOS,iOS,Android,MAC OS X,HTML,Englisch,OpenOffice / LibreOffice,Windows 7,Windows Vista,Windows NT/2000/XP,IT Consulting
jQuery,jQuery,JSON,Ajax,CSS,HTML,JavaScript,HTML5,GIT,XML/XSL,Englisch
knockout,knockout,ASP.NET WebAPI,Kendo UI,Windows Presentation Foundation (WPF),LINQ,ASP.NET MVC-Framework,Entity Framework,Team Foundation Server (TF Server),Angular Generalist,Windows Communication Foundation (WCF)


3. Create a list with the most simular skills of the corresponding known

In [32]:
similar_list = most_similar_to_rules_skills.values.tolist()
similar_list

[['.NET Framework',
  'LINQ',
  'NUnit',
  'Windows Forms',
  '.NET Compact Framework',
  'Entity Framework',
  'ASP.NET WebAPI',
  'ASP.NET',
  'Windows Presentation Foundation (WPF)',
  'Team Foundation Server (TF Server)'],
 ['ADO.NET',
  'BizTalk',
  'Windows Communication Foundation (WCF)',
  'Infragistics',
  '.NET Core',
  'Microsoft Enterprise Library',
  'Entity Framework',
  '.NET Compact Framework',
  'ASP.NET WebAPI',
  'ASP.NET MVC-Framework'],
 ['ASP.NET',
  'LINQ',
  'NUnit',
  'Entity Framework',
  'ASP.NET WebAPI',
  'Windows Forms',
  'Team Foundation Server (TF Server)',
  'ASP.NET MVC-Framework',
  'SQL',
  '.NET Framework'],
 ['ASP.NET MVC-Framework',
  'ASP.NET WebAPI',
  'LINQ',
  'Entity Framework',
  'NUnit',
  'NHibernate',
  'ASP.NET',
  'Windows Presentation Foundation (WPF)',
  'Jakarta Commons',
  'Kendo UI'],
 ['ASP.NET WebAPI',
  'Entity Framework',
  'LINQ',
  'NUnit',
  'ASP.NET MVC-Framework',
  'ASP.NET',
  'Team Foundation Server (TF Server)',
  '.N

4. Whatever happens here?

In [33]:
similar_list = list(set([item for sublist in similar_list for item in sublist]))
similar_list

['DevExpress',
 'Entwurfsmuster',
 'MS Exchange',
 'Node.js',
 'Express.js',
 'Dependency Injection',
 'Java',
 'Apache',
 'Tomcat',
 'iOS',
 'Statische Codeanalyse',
 'TeamCity',
 'JDBC',
 'Multiplattform/Multibrowser Testing',
 'RxJS',
 'PostgreSQL',
 'Objektrelationale Abbildung (ORM)',
 'Microsoft Access',
 '.NET Framework',
 'Softwaretests',
 'Windows 7',
 'Implementierung',
 'JUnit',
 'Oracle',
 'Gitlab CI',
 'Entity Framework',
 'Teamviewer',
 'PowerShell',
 'Debian',
 'HTML5',
 'Windows Communication Foundation (WCF)',
 'Google Cloud Platform',
 'jQuery',
 'Windows Server 2003',
 'NHibernate',
 'Microsoft SQL Server',
 'Azure DevOps',
 'Visual Studio',
 'VB.NET',
 'Ruby',
 'Android',
 'Agile Methoden',
 'Englisch',
 'Kendo UI',
 'Windows Presentation Foundation (WPF)',
 'C#',
 'Postfix',
 'Anforderungsanalyse',
 'samba',
 'Wasserfallmodel',
 'Vue.js',
 'Prozessmanagement',
 'Windows Identity Foundation (WIF)',
 'Internet Information Services (IIS)',
 'Qualitätsmanagement',
 'NG

5. The matrix which we produce is way smaller than in the approach without neighborhood

In [34]:
neighbourhood = data_matrix[similar_list].loc[similar_list]
neighbourhood

,DevExpress,Entwurfsmuster,MS Exchange,Node.js,Express.js,Dependency Injection,Java,Apache,Tomcat,iOS,...,GIMP,Google Android,JSON,Sport,Python,JSP,Redux,ActiveX,Continuous Integration (CI),CSS
DevExpress,1.000000,0.272631,0.182636,0.168476,0.000000,0.214734,0.171112,0.167646,0.138429,0.111940,...,0.212120,0.086183,0.219079,0.245649,0.166988,0.152751,0.000000,0.107540,0.176305,0.185411
Entwurfsmuster,0.272631,1.000000,0.243183,0.389339,0.048004,0.695924,0.555865,0.515367,0.526066,0.288047,...,0.315218,0.394763,0.595984,0.081106,0.403485,0.425492,0.108340,0.145946,0.692602,0.480464
MS Exchange,0.182636,0.243183,1.000000,0.187964,0.000000,0.213382,0.226718,0.410554,0.283838,0.310279,...,0.235083,0.188180,0.223383,0.000000,0.160600,0.198772,0.012877,0.270818,0.220471,0.201110
Node.js,0.168476,0.389339,0.187964,1.000000,0.268472,0.440251,0.358239,0.413713,0.289450,0.307999,...,0.339308,0.257539,0.548601,0.089137,0.320359,0.252107,0.294183,0.121415,0.411864,0.468705
Express.js,0.000000,0.048004,0.000000,0.268472,1.000000,0.041640,0.025840,0.015290,0.017950,0.000000,...,0.071483,0.022008,0.056803,0.000000,0.044502,0.000000,0.124524,0.000000,0.014457,0.131334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JSP,0.152751,0.425492,0.198772,0.252107,0.000000,0.464308,0.529540,0.504305,0.672010,0.241400,...,0.278349,0.283496,0.465522,0.098130,0.258037,1.000000,0.045304,0.106593,0.424579,0.395861
Redux,0.000000,0.108340,0.012877,0.294183,0.124524,0.145142,0.141276,0.111369,0.057028,0.099401,...,0.046630,0.008078,0.149949,0.000000,0.106510,0.045304,1.000000,0.000000,0.105968,0.243177
ActiveX,0.107540,0.145946,0.270818,0.121415,0.000000,0.143637,0.124116,0.136589,0.095333,0.134954,...,0.109561,0.163999,0.158906,0.000000,0.072102,0.106593,0.000000,1.000000,0.142376,0.114702
Continuous Integration (CI),0.176305,0.692602,0.220471,0.411864,0.014457,0.782950,0.565886,0.532819,0.508376,0.298040,...,0.365105,0.379211,0.683245,0.000000,0.388739,0.424579,0.105968,0.142376,1.000000,0.511744


6. Do The same cacluclation but within the smaller matrix

In [35]:
# A user vector containing only the neighbourhood items and
# the known user likes.
user_vector = data_skills_row_norm.iloc[user].loc[similar_list]

# Calculate the score.
score = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))

# Drop the known likes.
score = score.drop(known_user_skills)

7. Do some recommendations using the nneighbor approach

In [36]:
print(score.nlargest(5))

Angular Generalist    0.064874
Windows Forms         0.064454
Vue.js                0.064342
Kendo UI              0.064184
NHibernate            0.063726
dtype: float64


## Evaluate the neighborhood approach

## ___________________________________________________________________________________________

## Nearest Neighbors Approach

Employee-employee approach

Similarity between employees: due to dichotomous nature of data, Jaccard similarity is used for computing the similarity between employees. 

In [37]:
def jaccard_binary(x,y):
    """A function for finding the similarity between two binary vectors"""
    intersection = np.logical_and(x, y)
    union = np.logical_or(x, y)
    similarity = intersection.sum() / float(union.sum())
    return similarity

In [38]:
def compute_jaccard_sim(data,employee):
    sim = pd.DataFrame(index=[employee]) #dataframe 
    employee_data = data.loc[employee]
    sim_data = data.drop(employee,axis=0)
    for emp in sim_data.index:
        new_sim = pd.DataFrame(index=[employee],columns=[emp],data=jaccard_binary(employee_data,sim_data.loc[emp]))
        sim = pd.concat([sim,new_sim],axis=1)
    return sim       

In [39]:
def compute_n_nearest_neighbors(data,employee,n):
    neighbors = pd.DataFrame(index=[employee])
    similarities = compute_jaccard_sim(data,employee)
    for i in range(0,n):
        nearest_neighbor = similarities[similarities.idxmax(axis=1)]
        neighbors = pd.concat([neighbors,nearest_neighbor],axis=1)
        similarities.drop(nearest_neighbor,axis=1,inplace=True)
    return neighbors.columns.to_list()

In [40]:
def predict_skills_for_employee(data,employee,n_neighbors):
    neighbors = compute_n_nearest_neighbors(data,employee,n_neighbors)
    values_of_employee = data.loc[employee].to_numpy()
    neighbors_matrix = data.loc[neighbors].to_numpy()
    predicted_values = np.array([])
    for i in range(0,len(values_of_employee)):
        if values_of_employee[i] == 0: 
            predicted_values = np.append(predicted_values,
                                            # weighted averahe might be more accurate
                                            np.mean(neighbors_matrix[:,i]))
        else: 
            predicted_values = np.append(predicted_values,values_of_employee[i])
    return predicted_values

In [41]:
def predict_skills_for_matrix(data,n_neighbors): 
    # create emppy array of length according to columns (in order for later vstack to work)
    return_data = np.zeros(len(data.columns))
    for i in data.index: 
        single_predicition = predict_skills_for_employee(data,i,n_neighbors)
        return_data = np.vstack([return_data,single_predicition])
    # delete first entry (zeros) from return data
    return_data = return_data[1:len(return_data)]
    return pd.DataFrame(index=data.index,columns=data.columns,data=return_data)

In [42]:
# predict_skills_for_matrix(matrix,10)

## SVD approach from the suprise library on binary data 
* How to use SVD on binary data?
* How to combine it with the simularity matrix?
* Can we use the suprise lib on binary even it is not build for implict feedback?

In [43]:
NUM_TRIALS = 5
NUM_OUTER_SPLITS = 5

In [44]:
def make_ones_to_NaN_in_matrix(matrix):
    """A function for making the diagonal of a matrix to zero"""
    for i in range(len(matrix)):
        matrix[i,i] = np.NaN
    return pd.DataFrame(matrix)

In [45]:
matrix

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
matrix_reshaped = matrix.stack().reset_index()
matrix_reshaped.columns = ['user_id', 'item_id', 'rating']
matrix_reshaped.head(5)

,user_id,item_id,rating
0,0,.NET Compact Framework,0
1,0,.NET Core,0
2,0,.NET Framework,0
3,0,3D-Modellierung,0
4,0,ABAP,0


In [47]:
matrix_reshaped = matrix_reshaped.loc[matrix_reshaped['rating'] == 1]
matrix_reshaped.head(5)

,user_id,item_id,rating
22,0,Adobe Flash,1
30,0,Agile Methoden,1
35,0,Android,1
40,0,Angular Generalist,1
47,0,Apache,1


In [48]:
shuffled_index = np.random.RandomState(seed = 1).permutation(len(matrix_reshaped)) 
matrix_reshaped_shuffled=matrix_reshaped.iloc[shuffled_index, :]
matrix_reshaped_shuffled.shape

(24169, 3)

### Ml-Setup

In [49]:
reader = Reader(rating_scale=(0, 1))
svd_data = Dataset.load_from_df(matrix_reshaped_shuffled,reader)

In [50]:
def nested_cv(algo,data):
    
    start = time.time()
    rmse = np.zeros((NUM_TRIALS, NUM_TRIALS))
    mae = np.zeros((NUM_TRIALS, NUM_TRIALS))
    fit_times = np.zeros((NUM_TRIALS, NUM_TRIALS))
    test_times = np.zeros((NUM_TRIALS, NUM_TRIALS))

    for i in range(NUM_TRIALS):
        outer_cv = KFold(n_splits=NUM_OUTER_SPLITS,shuffle=True,random_state=36)
        cv_results = cross_validate(algo=algo,data=data,measures=["rmse","mae"],cv=outer_cv,n_jobs=8)
        rmse[i] = cv_results["test_rmse"]
        mae[i] = cv_results["test_mae"]
        fit_times[i] = cv_results["fit_time"]
        test_times[i] = cv_results["test_time"]

    return rmse,mae, fit_times, test_times

In [51]:
def add_result(results, name, accs, baccs, fit_times, test_times):
    '''
    Function adding the results returned by nested_cv to a dataframe. 
    Results will be aggregated for better comparison. 
    Parameters: 
    results = DataFrame which the results should be added to
    name = string describing the estimator which values are to be added
    accs = accuracy values of estimator
    baccs = balanced accuracy values of estimator 
    fit_times = fitting times of estimator
    test_times = testing times of estimator
    Output:
    results DataFrame containing an additional row
    '''
    row = pd.DataFrame({
        "name":name,
        "rmse_mean":accs.mean(), 
        "rmse_std":accs.std(), 
        "rmse_min":accs.min(), 
        "rmse_max":accs.max(), 
        "mae_mean":baccs.mean(), 
        "mae_std":baccs.std(), 
        "mae_min":baccs.min(), 
        "mae_max":baccs.max(), 
        "fit_time":fit_times.mean(),
        "test_time":test_times.mean()
        },index=[0])
    return pd.concat([results,row],ignore_index=True)

In [52]:
results = pd.DataFrame()

In [53]:
algo_list = [(BaselineOnly(),"Baseline"),
                (KNNBasic(),"k-NN"),
                (KNNWithMeans(),"Centered k-NN"),
                (SVD(),"SVD"),
                (SVDpp(),"SVD++"),
                (CoClustering(),"CoClustering"),
                (SlopeOne(),"SlopeOne"),
                (NMF(),"NMF")
            ]

In [54]:
for algo in algo_list:
    rmse, mae, fit_times, test_times = nested_cv(algo[0],svd_data)
    results = add_result(results,algo[1],rmse,mae,fit_times,test_times)

In [55]:
results

,name,rmse_mean,rmse_std,rmse_min,rmse_max,mae_mean,mae_std,mae_min,mae_max,fit_time,test_time
0,Baseline,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039055,0.074401
1,k-NN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.097736,0.996137
2,Centered k-NN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125023,1.099978
3,SVD,0.048570,0.000874,0.046469,0.050214,0.026824,0.000666,0.025069,0.028455,0.516777,0.085253
4,SVD++,0.018956,0.000423,0.018180,0.019738,0.010166,0.000309,0.009576,0.010685,5.604306,1.101898
5,CoClustering,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.839671,0.055173
6,SlopeOne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.165875,0.565887
7,NMF,0.060106,0.000149,0.059741,0.060483,0.059813,0.000116,0.059533,0.060093,0.864087,0.070971


## Predictions

In [56]:
svd = SVDpp(random_state= 1)
cv = cross_validate(svd, svd_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

KeyboardInterrupt: 

In [ ]:
trainset, testset = train_test_split(svd_data, test_size=0.25)
svd.fit(trainset)

In [ ]:
predictions = svd.test(testset)

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
recommendations  = get_top_n(predictions, n=10)
recommendations

defaultdict(list,
            {222: [('PhpStorm', 1),
              ('Jasmine', 1),
              ('Go', 1),
              ('XML/XSL', 1),
              ('Jenkins', 1),
              ('Test Driven Development (TDD)', 1),
              ('xtCommerce', 1),
              ('Touristik', 1),
              ('Fluid Templates', 1),
              ('Dreamweaver', 0.9930634788169571)],
             288: [('Log4J', 1),
              ('Mozilla FW', 1),
              ('JRun', 1),
              ('JIRA', 1),
              ('After Effects', 1),
              ('Cocoon', 1),
              ('Design Pattern', 1),
              ('C/C++', 1),
              ('Softwareeinführung', 1),
              ('Implementierung', 1)],
             80: [('Continuous Integration (CI)', 0.9967284382522603),
              ('Dienstleistungen', 0.9929672575090907),
              ('Anforderungsanalyse', 0.9916616854731911),
              ('Agile Methoden', 0.9908350727637881),
              ('Analytics', 0.9725679450018508),
     

In [ ]:
df_recommendations = pd.DataFrame(data= recommendations.items(), columns=['user_id', 'recommendations'])
df_recommendations


,user_id,recommendations
0,222,"[(PhpStorm, 1), (Jasmine, 1), (Go, 1), (XML/XS..."
1,288,"[(Log4J, 1), (Mozilla FW, 1), (JRun, 1), (JIRA..."
2,80,"[(Continuous Integration (CI), 0.9967284382522..."
3,169,"[(PHP/Horde, 1), (Hibernate, 1), (Ultra Edit, ..."
4,355,"[(Test Driven Development (TDD), 1), (Entwurfs..."
...,...,...
370,129,"[(JavaScript, 1)]"
371,370,"[(UX Beratung, 1)]"
372,147,"[(Prototyping, 1)]"
373,158,"[(Studentisches Praktikum, 1)]"


In [ ]:
df_recommendations.index = df_recommendations['user_id']
df_recommendations = df_recommendations.drop('user_id', axis=1)
df_recommendations

,recommendations
user_id,
222,"[(PhpStorm, 1), (Jasmine, 1), (Go, 1), (XML/XS..."
288,"[(Log4J, 1), (Mozilla FW, 1), (JRun, 1), (JIRA..."
80,"[(Continuous Integration (CI), 0.9967284382522..."
169,"[(PHP/Horde, 1), (Hibernate, 1), (Ultra Edit, ..."
355,"[(Test Driven Development (TDD), 1), (Entwurfs..."
...,...
129,"[(JavaScript, 1)]"
370,"[(UX Beratung, 1)]"
147,"[(Prototyping, 1)]"


In [ ]:
def display_recommendations_in_dataframe(data):
    skill = []
    score = []
    user = []
    for i in range(len(data)): # for each user
        for j in range(len(data.recommendations.iloc[i])):
            skill.append(data.recommendations.iloc[i][j][0])
            score.append(data.recommendations.iloc[i][j][1])
            user.append(data.index[i])
                
    df = pd.DataFrame({'user_id': user, 'skill': skill, 'score': score})
    return df

In [ ]:
final_recommendations = display_recommendations_in_dataframe(df_recommendations)
final_recommendations

,user_id,skill,score
0,222,PhpStorm,1.0
1,222,Jasmine,1.0
2,222,Go,1.0
3,222,XML/XSL,1.0
4,222,Jenkins,1.0
...,...,...,...
3178,129,JavaScript,1.0
3179,370,UX Beratung,1.0
3180,147,Prototyping,1.0
3181,158,Studentisches Praktikum,1.0


In [ ]:
final_recommendations.loc[(final_recommendations['user_id'] == 289)]

,user_id,skill,score
1052,289,Omni Commerce Connect (OCC) Rest APIs,1.000000
1053,289,VB/VBA,1.000000
1054,289,CSS,1.000000
1055,289,Design Pattern,1.000000
1056,289,Android,1.000000
1057,289,Promotion / Drools Engine,1.000000
1058,289,MySQL,1.000000
1059,289,Visual Studio,0.995330
1060,289,Softwaretests,0.989742
1061,289,FileZilla,0.989229


In [ ]:
# Looks like the user already has the recommended skills
matrix_reshaped.loc[(matrix_reshaped['user_id'] == 289)& (matrix_reshaped['item_id'] == "MySQL")]

,user_id,item_id,rating
212776,289,MySQL,1


### Nearest Neighbor Approach and SVD

In [ ]:
svd = SVDpp(random_state=1)
cv = cross_validate(svd, svd_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8116  0.8186  0.8151  0.8116  0.8167  0.8147  0.0028  
MAE (testset)     0.7929  0.7997  0.7967  0.7925  0.7990  0.7961  0.0030  
Fit time          3.13    2.21    2.21    2.81    2.54    2.58    0.36    
Test time         0.41    0.42    0.42    0.95    0.43    0.53    0.21    


In [ ]:
def jaccard_distance(x,y):
    """A function for finding the distance between two binary vectors"""
    intersection = np.logical_and(x, y)
    union = np.logical_or(x, y)
    distance = 1 - intersection.sum() / float(union.sum())
    return distance



In [ ]:
def compute_jaccard_distance_matrix(data):
    """A function for computing the jaccard distance matrix"""
    n = data.shape[0]
    distance_matrix = np.zeros((n,n)) #Creates a matrix of zeros. The shape is the number of rows and rows in the input data
    for i in range(n):
        for j in range(n):
            distance_matrix[i,j] = jaccard_binary(data.iloc[i,:],data.iloc[j,:])
    return distance_matrix

In [ ]:
jaccard_simularity_matrix  =compute_jaccard_distance_matrix(matrix)

In [ ]:
jaccard_simularity_matrix_df  = pd.DataFrame(jaccard_simularity_matrix)
jaccard_simularity_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,394,395,396,397,398,399,400,401,402,403
0,1.000000,0.336283,0.200000,0.186813,0.000000,0.000000,0.099010,0.299320,0.187500,0.300000,...,0.085106,0.074766,0.120879,0.134615,0.159722,0.049383,0.129412,0.023810,0.050000,0.306358
1,0.336283,1.000000,0.206897,0.166667,0.000000,0.000000,0.058824,0.212903,0.211382,0.233871,...,0.076087,0.178947,0.064516,0.150000,0.146853,0.064935,0.120482,0.037500,0.065789,0.218579
2,0.200000,0.206897,1.000000,0.319149,0.000000,0.000000,0.015625,0.124031,0.232558,0.122449,...,0.037037,0.029851,0.098039,0.180328,0.070796,0.054054,0.162791,0.000000,0.027027,0.139241
3,0.186813,0.166667,0.319149,1.000000,0.032258,0.032258,0.015625,0.132812,0.261905,0.089109,...,0.076923,0.061538,0.120000,0.125000,0.070796,0.054054,0.219512,0.000000,0.117647,0.125000
4,0.000000,0.000000,0.000000,0.032258,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.049383,0.064935,0.054054,0.054054,0.000000,0.000000,0.050000,0.025210,0.037500,0.060976,...,0.000000,0.069767,0.000000,0.088889,0.031579,1.000000,0.038462,0.000000,0.000000,0.032895
400,0.129412,0.120482,0.162791,0.219512,0.000000,0.000000,0.019231,0.108333,0.105882,0.076923,...,0.047619,0.017857,0.257143,0.132075,0.018692,0.038462,1.000000,0.000000,0.083333,0.083871
401,0.023810,0.037500,0.000000,0.000000,0.000000,0.000000,0.000000,0.016529,0.024390,0.000000,...,0.030303,0.093023,0.000000,0.020408,0.020619,0.000000,0.000000,1.000000,0.000000,0.012821
402,0.050000,0.065789,0.027027,0.117647,0.000000,0.000000,0.000000,0.034188,0.051282,0.061728,...,0.066667,0.022727,0.103448,0.043478,0.031915,0.000000,0.083333,0.000000,1.000000,0.026316


In [ ]:
jaccard_simularity_matrix_df =  make_ones_to_NaN_in_matrix(jaccard_simularity_matrix_df.to_numpy()) #jaccard_simularity_matrix.to_numpy()

In [ ]:
jaccard_simularity_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,394,395,396,397,398,399,400,401,402,403
0,NaN,0.336283,0.200000,0.186813,0.000000,0.000000,0.099010,0.299320,0.187500,0.300000,...,0.085106,0.074766,0.120879,0.134615,0.159722,0.049383,0.129412,0.023810,0.050000,0.306358
1,0.336283,NaN,0.206897,0.166667,0.000000,0.000000,0.058824,0.212903,0.211382,0.233871,...,0.076087,0.178947,0.064516,0.150000,0.146853,0.064935,0.120482,0.037500,0.065789,0.218579
2,0.200000,0.206897,NaN,0.319149,0.000000,0.000000,0.015625,0.124031,0.232558,0.122449,...,0.037037,0.029851,0.098039,0.180328,0.070796,0.054054,0.162791,0.000000,0.027027,0.139241
3,0.186813,0.166667,0.319149,NaN,0.032258,0.032258,0.015625,0.132812,0.261905,0.089109,...,0.076923,0.061538,0.120000,0.125000,0.070796,0.054054,0.219512,0.000000,0.117647,0.125000
4,0.000000,0.000000,0.000000,0.032258,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.049383,0.064935,0.054054,0.054054,0.000000,0.000000,0.050000,0.025210,0.037500,0.060976,...,0.000000,0.069767,0.000000,0.088889,0.031579,NaN,0.038462,0.000000,0.000000,0.032895
400,0.129412,0.120482,0.162791,0.219512,0.000000,0.000000,0.019231,0.108333,0.105882,0.076923,...,0.047619,0.017857,0.257143,0.132075,0.018692,0.038462,NaN,0.000000,0.083333,0.083871
401,0.023810,0.037500,0.000000,0.000000,0.000000,0.000000,0.000000,0.016529,0.024390,0.000000,...,0.030303,0.093023,0.000000,0.020408,0.020619,0.000000,0.000000,NaN,0.000000,0.012821
402,0.050000,0.065789,0.027027,0.117647,0.000000,0.000000,0.000000,0.034188,0.051282,0.061728,...,0.066667,0.022727,0.103448,0.043478,0.031915,0.000000,0.083333,0.000000,NaN,0.026316


In [ ]:
jaccard_simularity_matrix_df = jaccard_simularity_matrix_df.where(np.triu(np.ones(jaccard_simularity_matrix_df.shape)).astype(bool))

In [ ]:
jaccard_simularity_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,394,395,396,397,398,399,400,401,402,403
0,NaN,0.336283,0.200000,0.186813,0.000000,0.000000,0.099010,0.299320,0.187500,0.300000,...,0.085106,0.074766,0.120879,0.134615,0.159722,0.049383,0.129412,0.02381,0.050000,0.306358
1,NaN,NaN,0.206897,0.166667,0.000000,0.000000,0.058824,0.212903,0.211382,0.233871,...,0.076087,0.178947,0.064516,0.150000,0.146853,0.064935,0.120482,0.03750,0.065789,0.218579
2,NaN,NaN,NaN,0.319149,0.000000,0.000000,0.015625,0.124031,0.232558,0.122449,...,0.037037,0.029851,0.098039,0.180328,0.070796,0.054054,0.162791,0.00000,0.027027,0.139241
3,NaN,NaN,NaN,NaN,0.032258,0.032258,0.015625,0.132812,0.261905,0.089109,...,0.076923,0.061538,0.120000,0.125000,0.070796,0.054054,0.219512,0.00000,0.117647,0.125000
4,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.038462,0.00000,0.000000,0.032895
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.083333,0.083871
401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.012821
402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026316


Data structure for surprise library

In [ ]:
jaccard_simularity_matrix_df_reshaped = jaccard_simularity_matrix_df.stack().reset_index()
jaccard_simularity_matrix_df_reshaped.columns = ["user","neighbor","simularity"]

In [ ]:
jaccard_simularity_matrix_df_reshaped

,user,neighbor,simularity
0,0,1,0.336283
1,0,2,0.200000
2,0,3,0.186813
3,0,4,0.000000
4,0,5,0.000000
...,...,...,...
81401,400,402,0.083333
81402,400,403,0.083871
81403,401,402,0.000000
81404,401,403,0.012821


In [ ]:
shuffled_index = np.random.RandomState(seed = 1).permutation(len(jaccard_simularity_matrix_df_reshaped)) 
jaccard_simularity_matrix_df_reshaped=jaccard_simularity_matrix_df_reshaped.iloc[shuffled_index, :]

In [ ]:
reader = Reader()
svd_data = Dataset.load_from_df(jaccard_simularity_matrix_df_reshaped,reader)

In [ ]:
svd = SVD(random_state=1)
cv = cross_validate(svd, svd_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8731  0.8732  0.8744  0.8733  0.8725  0.8733  0.0006  
MAE (testset)     0.8643  0.8643  0.8656  0.8646  0.8636  0.8645  0.0006  
Fit time          0.99    0.86    0.87    0.89    0.84    0.89    0.05    
Test time         0.16    0.16    0.18    0.17    2.48    0.63    0.93    


In [ ]:
trainset, testset = train_test_split(svd_data, test_size=0.25)
svd.fit(trainset)
predictions = svd.test(testset)

Offen: Wie wird predicted?

In [ ]:
predictions[0]

Prediction(uid=17, iid=57, r_ui=0.12371134020618557, est=1, details={'was_impossible': False})

In [ ]:
def compute_jaccard_nearest_neighbors(data, k):
    """A function for computing the k nearest neighbors for each row in the data"""
    n = data.shape[0]
    nearest_neighbors = np.zeros((n,k)) #Creates a matrix of zeros. The shape is the number of rows and the number of nearest neighbors
    for i in range(n):
        nearest_neighbors[i,:] = data.iloc[i,:].sort_values().index[1:k+1]
    return nearest_neighbors

In [ ]:
def single_value_decomposition(data):
    """A function for performing single value decomposition on the data"""
    U, s, V = np.linalg.svd(data, full_matrices=False)
    return U, s, V

In [ ]:
U, s, V = single_value_decomposition(jaccard_nearest_neighbors_matrix)

## Implicit Library

### Basic usage

In [57]:
import implicit
import h5py
from implicit.datasets.movielens import get_movielens
from implicit.nearest_neighbours import (
    BM25Recommender,
    CosineRecommender,
    TFIDFRecommender,
    bm25_weight,
)
import scipy
from scipy.sparse import csr_matrix

c:\Users\Maximus\Envs\ds_ap\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=64) 

In [ ]:
data_skills_row_norm

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.000000,0.116248,0.000000,0.000000,0.116248,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.000000,0.179605,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.179605,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
400,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
401,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
402,0.0,0.377964,0.377964,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [ ]:
data_skills_row_norm_reshaped = data_skills_row_norm.stack().reset_index()
data_skills_row_norm_reshaped.columns = ['user_id', 'item_id', 'rating']
data_skills_row_norm_reshaped.head(5)

,user_id,item_id,rating
0,0,.NET Compact Framework,0.0
1,0,.NET Core,0.0
2,0,.NET Framework,0.0
3,0,3D-Modellierung,0.0
4,0,ABAP,0.0


In [ ]:
data_skills_row_norm_reshaped = data_skills_row_norm_reshaped.loc[(data_skills_row_norm_reshaped["rating"] != 0)]
data_skills_row_norm_reshaped

,user_id,item_id,rating
22,0,Adobe Flash,0.113961
30,0,Agile Methoden,0.113961
35,0,Android,0.113961
40,0,Angular Generalist,0.113961
47,0,Apache,0.113961
...,...,...,...
296894,403,XML/XSL,0.081923
296911,403,gradle,0.081923
296918,403,jQuery,0.081923
296924,403,npm,0.081923


In [ ]:
shuffled_index = np.random.RandomState(seed = 1).permutation(len(data_skills_row_norm_reshaped)) 
data_skills_row_norm_reshaped_shuffled=data_skills_row_norm_reshaped.iloc[shuffled_index, :]
data_skills_row_norm_reshaped_shuffled.head(5)

,user_id,item_id,rating
191356,260,ISTQB - Foundation Level,0.138675
13799,18,Statische Codeanalyse,0.093659
117706,160,Canvas,0.277350
216317,294,Google Android,0.121268
247061,336,CSS,0.107211


In [ ]:
data_skills_row_norm_reshaped_shuffled.dtypes

user_id      int64
item_id     object
rating     float64
dtype: object

In [ ]:
item_id_list = data_skills_row_norm_reshaped_shuffled.item_id.unique()

def make_dictonary_of_item_id_and_index(item_id_list):
    """A function for making a dictonary of item id and index"""
    item_id_index_dict = {}
    for index, item_id in enumerate(item_id_list):
        item_id_index_dict[item_id] = index
    return item_id_index_dict

In [ ]:
item_id_dict = make_dictonary_of_item_id_and_index(item_id_list)
item_id_dict

{'ISTQB - Foundation Level': 0,
 'Statische Codeanalyse': 1,
 'Canvas': 2,
 'Google Android': 3,
 'CSS': 4,
 'GIT': 5,
 'ftp-proxy': 6,
 'JIRA': 7,
 'Softwaredesign/-architektur': 8,
 'Hibernate': 9,
 'Qualitätsmanagement': 10,
 'Ant': 11,
 'Anforderungsanalyse': 12,
 'xtCommerce': 13,
 'nginx': 14,
 'CruiseControl': 15,
 'CVS': 16,
 'BPMN': 17,
 'xHTML': 18,
 'Log4J': 19,
 'f-prot': 20,
 'Continuous Integration (CI)': 21,
 'Angular Generalist': 22,
 'Englisch': 23,
 'OpenOffice / LibreOffice': 24,
 'TopLink': 25,
 'TCP/IP': 26,
 'Windows Communication Foundation (WCF)': 27,
 'Ultra Edit': 28,
 'Niederländisch': 29,
 'Node.js': 30,
 'Sun Solaris': 31,
 'Miro': 32,
 'HTML': 33,
 'WCMS': 34,
 'TestNG': 35,
 'Oracle XDK': 36,
 'SixCMS': 37,
 'MySQL': 38,
 'VB/VBA': 39,
 'Python': 40,
 'Windows Vista': 41,
 'SQL': 42,
 'Visual Studio': 43,
 'Mercurial': 44,
 'Public Key Infrastructures (PKI)': 45,
 'IntelliJ(Idea)': 46,
 'Windows Server 2008': 47,
 'Java EE': 48,
 'B2C': 49,
 'Azure DevOps

In [ ]:
data_skills_row_norm_reshaped_shuffled["item_id_index"] = data_skills_row_norm_reshaped_shuffled["item_id"].map(item_id_dict)
data_skills_row_norm_reshaped_shuffled.head(5)

C:\Users\Maximus\AppData\Local\Temp\ipykernel_10652\2410596155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_skills_row_norm_reshaped_shuffled["item_id_index"] = data_skills_row_norm_reshaped_shuffled["item_id"].map(item_id_dict)


,user_id,item_id,rating,item_id_index
191356,260,ISTQB - Foundation Level,0.138675,0
13799,18,Statische Codeanalyse,0.093659,1
117706,160,Canvas,0.277350,2
216317,294,Google Android,0.121268,3
247061,336,CSS,0.107211,4


In [ ]:
data_skills_row_norm_reshaped_shuffled_id = data_skills_row_norm_reshaped_shuffled[["user_id","item_id_index","rating"]]
data_skills_row_norm_reshaped_shuffled_id.head(5)

,user_id,item_id_index,rating
191356,260,0,0.138675
13799,18,1,0.093659
117706,160,2,0.277350
216317,294,3,0.121268
247061,336,4,0.107211


In [ ]:
data_skills_row_norm_reshaped_shuffled_id.index = data_skills_row_norm_reshaped_shuffled_id["user_id"]
data_skills_row_norm_reshaped_shuffled_id = data_skills_row_norm_reshaped_shuffled_id.drop(columns = ["user_id"])
data_skills_row_norm_reshaped_shuffled_id.head(5)

,item_id_index,rating
user_id,,
260,0,0.138675
18,1,0.093659
160,2,0.277350
294,3,0.121268
336,4,0.107211


In [ ]:
len(data_skills_row_norm_reshaped_shuffled_id)

24169

In [ ]:
data_skills_row_norm_reshaped_shuffled_id_csr = scipy.sparse.csr_matrix(data_skills_row_norm_reshaped_shuffled_id.values)

In [59]:
data_skills_row_norm_csr = scipy.sparse.csr_matrix(data_skills_row_norm.values)


In [62]:
data_skills_row_norm_csr?

Type:        csr_matrix
String form:
(0, 22)	0.11396057645963795
           (0, 30)	0.11396057645963795
           (0, 35)	0.11396057645963795
           (0, 4 <...>  (403, 713)	0.08192319205190406
           (403, 719)	0.08192319205190406
           (403, 730)	0.08192319205190406
File:        c:\users\maximus\envs\ds_ap\lib\site-packages\scipy\sparse\_csr.py
Docstring:  
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standar

In [64]:
# train the model on a sparse matrix of user/item/confidence weights
model.fit(data_skills_row_norm_csr)

100%|██████████| 15/15 [00:02<00:00,  6.67it/s]


In [ ]:
model.fit?

Signature: model.fit(user_items, show_progress=True, callback=None)
Docstring:
Factorizes the user_items matrix.

After calling this method, the members 'user_factors' and 'item_factors' will be
initialized with a latent factor model of the input data.

The user_items matrix does double duty here. It defines which items are liked by which
users (P_ui in the original paper), as well as how much confidence we have that the user
liked the item (C_ui).

The negative items are implicitly defined: This code assumes that positive items in the
user_items matrix means that the user liked the item. The negatives are left unset in this
sparse matrix: the library will assume that means Piu = 0 and Ciu = 1 for all these items.
Negative items can also be passed with a higher confidence value by passing a negative
value, indicating that the user disliked the item.

Parameters
----------
user_items: csr_matrix
    Matrix of confidences for the liked items. This matrix should be a csr_matrix where
    

In [79]:
userid= 13
dict_user_skills = {}
for value, name in zip(data_skills_row_norm.iloc[userid,:], data_skills_row_norm.columns):
    if value > 0:
        dict_user_skills[name] = value
    else:
        next

dict_user_skills

{'Android': 0.23570226039551587,
 'Apache': 0.23570226039551587,
 'GIT': 0.23570226039551587,
 'JavaScript': 0.23570226039551587,
 'Joomla': 0.23570226039551587,
 'Microsoft SQL Server': 0.23570226039551587,
 'MySQL': 0.23570226039551587,
 'Oracle': 0.23570226039551587,
 'PHP': 0.23570226039551587,
 'PhpStorm': 0.23570226039551587,
 'Pimcore': 0.23570226039551587,
 'SUSE Linux': 0.23570226039551587,
 'Subversion': 0.23570226039551587,
 'Symfony': 0.23570226039551587,
 'Ubuntu': 0.23570226039551587,
 'Windows 10': 0.23570226039551587,
 'Windows 7': 0.23570226039551587,
 'Windows NT/2000/XP': 0.23570226039551587}

In [85]:
skill_ids, scores  = model.recommend(userid, data_skills_row_norm_csr[userid], N = 20 , filter_already_liked_items=False)
recommendations_df = pd.DataFrame({"skill": data_skills_row_norm.columns[skill_ids], "score": scores, "already_liked": np.in1d(skill_ids, data_skills_row_norm_csr[userid].indices)})
recommendations_df

,skill,score,already_liked
0,PHP,1.278322,True
1,Windows 10,1.011779,True
2,PhpStorm,0.957290,True
3,Microsoft SQL Server,0.740192,True
4,Windows NT/2000/XP,0.727629,True
5,Windows 7,0.702665,True
6,GIT,0.682084,True
7,Apache,0.647216,True
8,Symfony,0.637661,True
9,MySQL,0.637475,True


In [88]:
skill_and_categories = df[["skill","category"]].drop_duplicates()

In [89]:
recommendations_df.merge(skill_and_categories, how = "left", left_on = "skill", right_on = "skill")

,skill,score,already_liked,category
0,PHP,1.278322,True,Webentwicklung
1,Windows 10,1.011779,True,Betriebssystem
2,PhpStorm,0.957290,True,Umgebungen
3,Microsoft SQL Server,0.740192,True,Datenbank
4,Windows NT/2000/XP,0.727629,True,Betriebssystem
5,Windows 7,0.702665,True,Betriebssystem
6,GIT,0.682084,True,"CI/CD, Build- und Versionskontrollsysteme"
7,Apache,0.647216,True,Applikationsserver
8,Symfony,0.637661,True,PHP
9,MySQL,0.637475,True,Datenbank


In [92]:
# Id and skills
dict_skills_id = {}
for c,skill in enumerate(data_skills_row_norm.columns):
        dict_skills_id[c] = skill

dict_skills_id

{0: '.NET Compact Framework',
 1: '.NET Core',
 2: '.NET Framework',
 3: '3D-Modellierung',
 4: 'ABAP',
 5: 'ADO.NET',
 6: 'AIX',
 7: 'ARIS',
 8: 'ARIS ITArchitect',
 9: 'AS400',
 10: 'ASP Generalist',
 11: 'ASP.NET',
 12: 'ASP.NET MVC-Framework',
 13: 'ASP.NET WebAPI',
 14: 'AWS',
 15: 'AWS Lambda',
 16: 'Abstract',
 17: 'Access',
 18: 'Accessibility / WCAG',
 19: 'Active Directory',
 20: 'ActiveX',
 21: 'Adobe CC',
 22: 'Adobe Flash',
 23: 'Adobe Illustrator',
 24: 'Adobe InDesign',
 25: 'Adobe Indesign',
 26: 'Adobe Photoshop',
 27: 'Adobe Premiere',
 28: 'Adobe XD',
 29: 'After Effects',
 30: 'Agile Methoden',
 31: 'Ajax',
 32: 'Alexa-Skills',
 33: 'Alpine',
 34: 'Analytics',
 35: 'Android',
 36: 'Android Studio',
 37: 'Anforderungsanalyse',
 38: 'Anforderungsmanagement',
 39: 'Angular (2 und höher)',
 40: 'Angular Generalist',
 41: 'Angular Material',
 42: 'Angular Theming',
 43: 'AngularJS',
 44: 'Animations (transition, @keyframes)',
 45: 'Ansible',
 46: 'Ant',
 47: 'Apache',
 4

In [99]:
keys = [k for k, v in dict_skills_id.items() if v == 'Python']
print(keys)

[458]


In [101]:
# find related item
skill_id, sim = model.similar_items(0, N=10)
# display the results using pandas for nicer formatting
simularity_df  = pd.DataFrame({"skill": data_skills_row_norm.columns[skill_id], "score": sim})
simularity_df.merge(skill_and_categories, how = "left", left_on = "skill", right_on = "skill")

,skill,score,category
0,.NET Compact Framework,1.000000,.NET Frameworks und Tools
1,ADO.NET,0.664472,.NET Frameworks und Tools
2,DevExpress,0.648678,.NET Frameworks und Tools
3,Silverlight,0.632325,.NET Frameworks und Tools
4,Windows Communication Foundation (WCF),0.631332,.NET Frameworks und Tools
5,ASP Generalist,0.606654,.NET Frameworks und Tools
6,Microsoft Enterprise Library,0.597184,.NET Frameworks und Tools
7,Windows Presentation Foundation (WPF),0.578164,.NET Frameworks und Tools
8,Windows Forms,0.562298,.NET Frameworks und Tools
9,AS400,0.557862,Betriebssystem


## Rest

In [ ]:
employee_dict = {}
for n in range(1,data["person"].nunique()+1):
    employee_dict[n] = data["person"].unique()[n-1]

In [ ]:
# So later functions work, it makes sense to rename persons from 1 to maxno. 
# Create dictionary which matches the employee
persons = data["person"].unique()
#np.arange(1,len(data["person"].unique())+1)
skills = sorted(data["skill"].unique())

In [ ]:
skills_per_person = []
for person in data["person"].unique(): 
    skillset = []
    for skill in skills:
        if skill in data[data["person"] == person]["skill"].unique():
            skillset.append(1)
        else:
            skillset.append(0)
    skills_per_person.append(skillset)


KeyboardInterrupt: 

In [ ]:
matrix = pd.DataFrame(index=persons,columns=skills,data=skills_per_person)

In [ ]:
matrix

,.NET Compact Framework,.NET Core,.NET Framework,3D-Modellierung,ABAP,ADO.NET,AIX,ARIS,ARIS ITArchitect,AS400,...,ramda.js,ranorex,samba,script.aculo.us,varnish,visual paradigm,vnc,vs code,xHTML,xtCommerce
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1479,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def calculate_cosine_similarity(data):
    # create empty matrix
    similarity_matrix = np.zeros((data.shape[0], data.shape[0]))
    # iterate over rows
    for i in range(data.shape[0]):
        # iterate over rows again
        for j in range(data.shape[0]):
            # calculate cosine similarity
            similarity_matrix[i,j] = np.dot(data.iloc[i],data.iloc[j]) / (magnitude[i] * magnitude[j])
    return similarity_matrix

In [ ]:
test_matrix = calculate_cosine_similarity(data_skills)

In [ ]:
pd.DataFrame(test_matrix)

,0,1,2,3,4,5,6,7,8,9,...,394,395,396,397,398,399,400,401,402,403
0,1.000000,0.503411,0.368423,0.347955,0.000000,0.000000,0.195441,0.469629,0.315817,0.461576,...,0.182337,0.147895,0.250713,0.249167,0.276287,0.161165,0.287588,0.075974,0.172292,0.494809
1,0.503411,1.000000,0.375816,0.313180,0.000000,0.000000,0.119618,0.359290,0.349001,0.379288,...,0.162747,0.320584,0.139497,0.272322,0.257325,0.205499,0.266690,0.116248,0.219687,0.380935
2,0.368423,0.375816,1.000000,0.483871,0.000000,0.000000,0.030802,0.269145,0.414781,0.242486,...,0.071842,0.058272,0.179605,0.308546,0.151456,0.127000,0.288430,0.000000,0.067884,0.323704
3,0.347955,0.313180,0.483871,1.000000,0.179605,0.179605,0.030802,0.285967,0.456259,0.181865,...,0.143684,0.116543,0.215526,0.224397,0.151456,0.127000,0.370839,0.000000,0.271538,0.294277
4,0.000000,0.000000,0.000000,0.179605,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.161165,0.205499,0.127000,0.127000,0.000000,0.000000,0.121268,0.099340,0.122474,0.198889,...,0.000000,0.172062,0.000000,0.220863,0.111803,1.000000,0.081111,0.000000,0.000000,0.144821
400,0.287588,0.266690,0.288430,0.370839,0.000000,0.000000,0.039344,0.279328,0.238416,0.180679,...,0.091766,0.037216,0.412948,0.250801,0.048365,0.081111,1.000000,0.000000,0.173422,0.244328
401,0.075974,0.116248,0.000000,0.000000,0.000000,0.000000,0.000000,0.062439,0.076980,0.000000,...,0.066667,0.216295,0.000000,0.052058,0.070273,0.000000,0.000000,1.000000,0.000000,0.054615
402,0.172292,0.219687,0.067884,0.271538,0.000000,0.000000,0.000000,0.141598,0.174574,0.212622,...,0.151186,0.061314,0.226779,0.118056,0.119523,0.000000,0.173422,0.000000,1.000000,0.123856


# Notizen
* Evt Überlegen, ob wir die Kategorien wie Betriebssysteme usw. im ersten Lauf noch beibehalten sollten


Drop certain skill categories: 